<h2><b>Langchain : Agent</b></h2>

In [1]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper = WikipediaAPIWrapper(top_k_results=1 , doc_content_chars_max=200)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)

wiki.name

'wikipedia'

In [2]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_google_genai.embeddings import GoogleGenerativeAIEmbeddings
from dotenv import load_dotenv
from langchain_text_splitters import RecursiveCharacterTextSplitter

load_dotenv()

loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()

documents = RecursiveCharacterTextSplitter(chunk_size=1000 , chunk_overlap=200).split_documents(docs)

vectordb = FAISS.from_documents(documents[:100],GoogleGenerativeAIEmbeddings(model="models/embedding-001"))
retriever = vectordb.as_retriever()

retriever

USER_AGENT environment variable not set, consider setting it to identify your requests.


VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001E3DB85DE80>, search_kwargs={})

In [3]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(
    retriever,
    "Langsmith_Search",
    "Search for the information about LangSmith, For any Question about Langsmith , you must have be using this tool"
)

retriever_tool.name

'Langsmith_Search'

In [4]:
#Arxiv Tool- Website where all the Research papers Uploaded
from langchain_community.tools import ArxivQueryRun
from langchain_community.utilities import ArxivAPIWrapper

arxiv_wrapper = ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=200)
arxiv = ArxivQueryRun(arxiv_wrapper = arxiv_wrapper)
arxiv.name

'arxiv'

In [5]:
tools=[wiki,arxiv,retriever_tool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'd:\\New Folder\\Main\\Coding\\Python\\Data Science\\Projects\\Practice\\GenAI\\langchain Campusx\\langcamp\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=3, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=4000)),
 Tool(name='Langsmith_Search', description='Search for the information about LangSmith, For any Question about Langsmith , you must have be using this tool', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000001E3CAC5EA20>, retriever=Vec

In [6]:
from dotenv import load_dotenv
load_dotenv()

# from langchain_google_genai.chat_models import ChatGoogleGenerativeAI
# llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro-latest")

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo")

# from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
# from dotenv import load_dotenv

# load_dotenv()

# model = HuggingFaceEndpoint(
#     repo_id="google/flan-t5-large",  # Safe model
#     task="text2text-generation"
# )
# llm = ChatHuggingFace(llm=model)


In [7]:
from langchain import hub

# prompt = hub.pull("hwchase17/structured-chat-agent")
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

d:\New Folder\Main\Coding\Python\Data Science\Projects\Practice\GenAI\langchain Campusx\langcamp\Lib\site-packages\langsmith\client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [ ]:
# Agents
from langchain.agents import create_structured_chat_agent ,create_openai_functions_agent

# agent =create_structured_chat_agent(llm,tools,prompt)
agent = create_openai_functions_agent(llm,tools,prompt)
# agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_function_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.mess

In [ ]:
#Agent Executer
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent,tools=tools,verbose=True)
# agent_executor

AgentExecutor(verbose=True, agent=RunnableAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_function_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='H

In [10]:
# agent_executor.invoke({"input":"Tell me about Langsmith"})

In [11]:
agent_executor.invoke({"input":"Tell me about Attention all you need Research Paper"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': 'Attention all you need Research Paper'}`


Published: 2023-03-02
Title: Methods used in nanostructure modeling
Authors: I. Camps
Summary: How many times you need to change your method description because you were
"accused" of plagiarism from text you already published? I will use this
preprint to add all the methods I currently used in running the simulations for
my research works. Then, I will cite it as needed.

Published: 2022-07-05
Title: Betti numbers of attention graphs is all you really need
Authors: Laida Kushnareva, Dmitri Piontkovski, Irina Piontkovskaya
Summary: We apply methods of topological analysis to the attention graphs, calculated
on the attention heads of the BERT model ( arXiv:1810.04805v2 ). Our research
shows that the classifier built upon basic persistent topological features
(namely, Betti numbers) of the trained neural network can achieve
classification results on par with the conventio

{'input': 'Tell me about Attention all you need Research Paper',
 'output': 'I found information on three research papers related to the phrase "Attention all you need." Here are the details:\n\n1. Published: 2023-03-02\n   Title: Methods used in nanostructure modeling\n   Author: I. Camps\n   Summary: This paper discusses the methods used in nanostructure modeling for research works involving simulations.\n\n2. Published: 2022-07-05\n   Title: Betti numbers of attention graphs is all you really need\n   Authors: Laida Kushnareva, Dmitri Piontkovski, Irina Piontkovskaya\n   Summary: The research applies topological analysis methods to attention graphs derived from the BERT model. It shows that using basic persistent topological features (Betti numbers) can achieve classification results similar to conventional methods in Natural Language Processing.\n\n3. Published: 2024-11-14\n   Title: Grounding is All You Need? Dual Temporal Grounding for Video Dialog\n   Authors: You Qin, Wei Ji, X

In [ ]:
# agent_executor.invoke({"input":"Tell me about the History of India"})

In [13]:
# agent_executor.invoke({"input":"Tell me about Machine learning"})